In [1]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [1]:
import time
import wikipedia
from functools import lru_cache
from _pipeline import create_payload, model_req

# Function to retrieve data from Wikipedia with multiple summaries
def retrieve_wikipedia_data(query):
    try:
        results = wikipedia.search(query)  # Get relevant article titles
        summaries = []
        for title in results[:3]:  # Limit to top 3 results
            try:
                summary = wikipedia.summary(title, sentences=5)
                summaries.append(f"**{title}**: {summary}")
            except Exception:
                continue  # Skip problematic pages
        return "\n\n".join(summaries) if summaries else "No relevant Wikipedia data found."
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Multiple possible matches found: {e.options}. Please refine your query."
    except wikipedia.exceptions.HTTPError:
        return "Sorry, there was an issue accessing Wikipedia."
    except wikipedia.exceptions.RedirectError:
        return "Sorry, the page is a redirect. Try a more specific query."
    except wikipedia.exceptions.Timeout:
        return "The request timed out. Please try again later."
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

# Define the inbound prompt for requirement analysis
RAG_SHOT = """You are a highly capable AI assistant specialized in automating requirement analysis.  
Your task is to extract and analyze requirements for an AI-powered Study Companion Discord Bot by retrieving relevant data from Wikipedia and generating structured, insightful answers.

Please ensure your answers clearly outline key features, technical specifications, and best practices based on the retrieved data.
"""

PROMPT = RAG_SHOT + "\nAnalyze the following requirement: {user_query}"

# Cache frequent Wikipedia data retrievals to improve performance
@lru_cache(maxsize=50)
def get_retrieved_data(query):
    return retrieve_wikipedia_data(query)

# Function to format the response with new lines between sentences
def format_response(response):
    sentences = response.split('. ')
    formatted_response = '\n'.join([sentence.strip() + '.' for sentence in sentences if sentence])
    return formatted_response

def generate_requirement_analysis(user_query):
    retrieved_wiki = get_retrieved_data(user_query)
    
    # Incorporate the retrieved data into the prompt
    prompt = PROMPT.format(user_query=user_query) + "\n\n" + retrieved_wiki
    
    # Configure payload for the AI model request
    payload = create_payload(
        target="ollama",                
        model="llama3.2:latest",        
        prompt=prompt,
        temperature=0.5,       # More deterministic output
        max_tokens=700,        # More detailed responses for requirement analysis
        top_p=0.85,            # More refined token selection
        top_k=50,              # Restrict token selection further
        repetition_penalty=1.15,  # Penalize repetitions
        num_ctx=9096,          # Context window size
    )
    
    # Send the request to the model
    start_time = time.time()
    response = model_req(payload=payload)
    end_time = time.time()
    print(f'Time taken: {end_time - start_time}s')
    
    # Extract response text (assuming response is a tuple with the second element as the text)
    response_text = response[1] if isinstance(response, tuple) else response
    
    # Format the response for readability
    formatted_response = format_response(response_text)
    return formatted_response

# Example usage:
if __name__ == "__main__":
    user_query = "What are the key requirements for an AI-powered Study Companion Discord Bot?"
    response = generate_requirement_analysis(user_query)
    print(response)


{'model': 'llama3.2:latest', 'prompt': 'You are a highly capable AI assistant specialized in automating requirement analysis.  \nYour task is to extract and analyze requirements for an AI-powered Study Companion Discord Bot by retrieving relevant data from Wikipedia and generating structured, insightful answers.\n\nPlease ensure your answers clearly outline key features, technical specifications, and best practices based on the retrieved data.\n\nAnalyze the following requirement: What are the key requirements for an AI-powered Study Companion Discord Bot?\n\n**Comparison of user features of messaging platforms**: Comparison of user features of messaging platforms refers to a comparison of all the various user features of various electronic instant messaging platforms. This includes a wide variety of resources; it includes standalone apps, platforms within websites, computer software, and various internal functions available on specific devices, such as iMessage for iPhones.\nThis entr